In [1]:
# Dependencies
import pandas as pd
import requests as req
import time
from bs4 import BeautifulSoup as bs
from splinter import Browser
from selenium import webdriver

In [2]:
news_data = {} 
paragraph_text = []

In [3]:
base_url = "https://mars.nasa.gov/"
nasa_url = "https://mars.nasa.gov/news/" 
response_1 = req.get(nasa_url)   
nasa_soup = bs(response_1.text, 'html.parser') 

In [4]:
soup_div = nasa_soup.find(class_="slide")
soup_news = soup_div.find_all('a')
news_title = soup_news[1].get_text().strip()

In [5]:
soup_p = soup_div.find_all('a', href=True)                                   
soup_p_url = soup_p[0]['href']                                              
paragraph_url = base_url + soup_p_url                                       
response_2 = req.get(paragraph_url)                                         
para_soup = bs(response_2.text, "html.parser")                               
ww_paragraphs = para_soup.find(class_='wysiwyg_content')                    
paragraphs = ww_paragraphs.find_all('p')    

In [6]:
for paragraph in paragraphs:                                                 
    clean_paragraph = paragraph.get_text().strip()                              
    paragraph_text.append(clean_paragraph)      

In [7]:
news_data["news_title"] = news_title
news_data["paragraph_text_1"] = paragraph_text[0]
news_data["paragraph_text_2"] = paragraph_text[1]
news_data  

{'news_title': 'NASA Invites Students to Name Mars 2020 Rover',
 'paragraph_text_1': 'Red rover, red rover, send a name for Mars 2020 right over! NASA is recruiting help from students nationwide to find a name for its next Mars rover mission.',
 'paragraph_text_2': 'Starting Tuesday, Aug. 27, K-12 students in U.S. public, private and home schools can enter the Mars 2020 Name the Rover essay contest. One grand prize winner will name the rover and be invited to see the spacecraft launch in July 2020 from Cape Canaveral Air Force Station in Florida.'}

In [10]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
jpl_fullsize_url = 'https://photojournal.jpl.nasa.gov/jpeg/'                 # defines base URL for fullsize images
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"     

In [11]:
browser.visit(jpl_url)                                                       
jpl_html = browser.html                                                      
jpl_soup = bs(jpl_html, 'html.parser')                                       

In [12]:
featured_image_list = []

for image in jpl_soup.find_all('div',class_="img"):                          # extracts all images
    featured_image_list.append(image.find('img').get('src'))

In [13]:
feature_image = featured_image_list[0]
temp_list_1 = feature_image.split('-')
temp_list_2 = temp_list_1[0].split('/')
featured_image_url = jpl_fullsize_url + temp_list_2[-1] + '.jpg'
featured_image_url    

'https://photojournal.jpl.nasa.gov/jpeg/PIA23579.jpg'

In [14]:
browser.quit() 

In [15]:
### MARS WEATHER

In [16]:
browser = Browser('chrome', headless=False)                                  
tweet_url = 'https://twitter.com/marswxreport?lang=en'                      
browser.visit(tweet_url)

In [17]:
tweet_html = browser.html
tweet_soup = bs(tweet_html, 'html.parser')

In [18]:
weather_info_list = []                                                       


for weather_info in tweet_soup.find_all('p',class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text"):
    weather_info_list.append(weather_info.text.strip())                     


In [19]:
for value in reversed(weather_info_list):
    if value[:3]=='Sol':                                                    
        mars_weather = value   

In [20]:
mars_weather

NameError: name 'mars_weather' is not defined

In [21]:
browser.quit() 

In [22]:
## MARS FACTS

In [23]:
facts_url = 'https://space-facts.com/mars/' 

In [24]:
fact_list = pd.read_html(facts_url)

In [25]:
facts_df = fact_list[0]

In [26]:
facts_table = facts_df.to_html(header=False, index=False)                    
print(facts_table)     

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [27]:
## MARS HEMISPHERES

In [28]:
browser = Browser('chrome', headless=False)                                                                   
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(usgs_url)                                                     

In [29]:
usgs_html = browser.html                                                     
usgs_soup = bs(usgs_html, 'html.parser')                                     

In [30]:
hemisphere_image_urls = []                                                   # Creates empty list

products = usgs_soup.find('div', class_='result-list')                       # finds products
hemispheres = products.find_all('div', class_='item')                        # finds hemispheres

for hemisphere in hemispheres:                                               # iterates through hemispheres
    title = hemisphere.find('div', class_='description')
    
    title_text = title.a.text                                                # extracts cleaned title
    title_text = title_text.replace(' Enhanced', '')
    browser.click_link_by_partial_text(title_text)                           # (automated) click
    
    usgs_html = browser.html                                                 # acquires response from URL
    usgs_soup = bs(usgs_html, 'html.parser')                                 # sends response to beautiful soup
    
    image = usgs_soup.find('div', class_='downloads').find('ul').find('li')  # extracts image url
    img_url = image.a['href']
    
    hemisphere_image_urls.append({'title': title_text, 'img_url': img_url})  # adds dictionary to list  
    
    browser.click_link_by_partial_text('Back')                               # (automated) click back


C:\Users\sedra\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:504: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


ElementDoesNotExist: no elements could be found with link by partial text "Back"

In [31]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}]

In [32]:
browser.quit()